**Общая информация**

**Срок сдачи:** 21 октября 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [303]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier, KDTree
from sklearn.metrics import accuracy_score, pairwise_distances, make_scorer
from sklearn.feature_extraction.text import CountVectorizer as cntvect
from sklearn.preprocessing import MaxAbsScaler

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [304]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute', metric='euclidean'):
        self.neighbours = n_neighbors
        self.alg = algorithm
        self.metr = metric
        self.scaler = MaxAbsScaler()
    
    def fit(self, X, Y):
        self.Y_train = Y
        self.X_train = self.scaler.fit_transform(X, Y)
        
    def predict_brute(self, X):
      X = self.scaler.transform(X)
      Y_pred = []
      n_ind = pairwise_distances(X, self.X_train).argsort(axis=1)[:, :self.neighbours]
      n_arr = self.Y_train[n_ind]
      for elem in n_arr:
        neigh, num = np.unique(elem, axis=0, return_counts = True)
        num = num.argsort()
        Y_pred.append(neigh[num[-1]])

      return Y_pred

    def predict_tree(self, X):
        X = self.scaler.transform(X)
        Y_pred = []
        self.X_train = KDTree(X)
        t, n_ind = self.X_train.query(X, k = self.neighbours)
        n_arr = self.Y_train[n_ind]
        for elem in n_arr:
          neigh, num = np.unique(elem, axis=0, return_counts = True)
          num = num.argsort()
          Y_pred.append(neigh[num[-1]])

        return Y_pred

    def predict(self, X):
      if self.alg == 'brute':
        return self.predict_brute(X)
      if self.alg == 'kd_tree':
        return self.predict_tree(X)
    

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [332]:
iris = datasets.load_iris()

In [333]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [334]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [335]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [348]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
print(accuracy_score(y_test, my_clf_pred))
print(accuracy_score(y_test, sklearn_pred ))
assert accuracy_score(y_test, my_clf_pred) > accuracy_score(y_test, sklearn_pred) or abs(accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred) )<0.005, "Score must be simillar or better"

0.8666666666666667
0.8666666666666667


**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [311]:
%time clf.fit(X_train, y_train)

CPU times: user 1.11 ms, sys: 0 ns, total: 1.11 ms
Wall time: 905 µs


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [312]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.21 ms, sys: 26 µs, total: 1.24 ms
Wall time: 5.08 ms


In [313]:
%time clf.predict(X_test)

CPU times: user 2.95 ms, sys: 0 ns, total: 2.95 ms
Wall time: 2.79 ms


array([1, 0, 2, 1, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0])

In [314]:
%time my_clf.predict(X_test)

CPU times: user 2.2 ms, sys: 11 µs, total: 2.21 ms
Wall time: 2.23 ms


[1, 0, 2, 1, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0]

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [315]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [316]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [317]:
%time clf.fit(X_train, y_train)

CPU times: user 1.41 ms, sys: 0 ns, total: 1.41 ms
Wall time: 1.3 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [318]:
%time my_clf.fit(X_train, y_train)

CPU times: user 803 µs, sys: 0 ns, total: 803 µs
Wall time: 667 µs


In [319]:
%time clf.predict(X_test)

CPU times: user 1.84 ms, sys: 850 µs, total: 2.69 ms
Wall time: 2.61 ms


array([2, 1, 1, 1, 0, 1, 1, 2, 0, 2, 1, 0, 0, 2, 0])

In [320]:
%time my_clf.predict(X_test)

CPU times: user 2.16 ms, sys: 1.04 ms, total: 3.21 ms
Wall time: 2.82 ms


[1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2]

In [346]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
print(accuracy_score(y_test, my_clf_pred))
print(accuracy_score(y_test, sklearn_pred ))
assert accuracy_score(y_test, my_clf_pred) > accuracy_score(y_test, sklearn_pred) or abs(accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred) )<0.005, "Score must be simillar or better"

0.8666666666666667
0.8666666666666667


**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [338]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [339]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [340]:
vocab = cntvect(max_df=0.995, min_df=0.005).fit_transform(data)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [342]:
folds = KFold(n_splits=3, shuffle=True, random_state=100500)

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [343]:
def opt_neighb(vocab, target, folds, metr = 'euclidean'):
  scores = []
  for i in range(1,21):
    score = cross_val_score(MyKNeighborsClassifier(n_neighbors=i, algorithm='brute'), vocab, target, cv=KFold(n_splits=3, shuffle=True, random_state=100500), scoring=make_scorer(accuracy_score))
    mean_score = np.mean(score)
    scores.append(mean_score)
  return scores

In [344]:
scores = opt_neighb(vocab, target, folds)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


ValueError: ignored

In [ ]:
plt.plot(range(1,21), scores)
plt.show()

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

In [ ]:
scores2 = opt_neighb(vocab, target, folds, 'cosine')
plt.plot(range(1,21), scores2)
plt.show()

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])
test_data = newsgroups['data']
test_target = newsgroups['target']

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.